# Importando as biblitecas

In [1]:
import pandas as pd
import re
import numpy as np
import folium

# Lendo o arquivo de mortalidade

In [2]:
Mortalidade_2019 = pd.read_csv(r'C:\Projeto Final\data\Mortalidade_Geral_2019.csv', sep = ';')

C:\Users\Leonardo\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Estudando a tabela 

In [3]:
Mortalidade_2019.set_index('CONTADOR')

,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,DTCONINV,FONTES,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA
CONTADOR,,,,,,,,,,,,,,,,,,,,,
1,1,2,21012019,540.0,829.0,290750.0,18021961.0,457,1,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2,28012019,1630.0,829.0,290070.0,3071938.0,480,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2,27012019,2320.0,829.0,290750.0,15111945.0,473,2,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,2,12012019,2300.0,829.0,290750.0,4111971.0,447,1,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,2,10012019,1726.0,829.0,292520.0,12101946.0,472,2,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1378902,1,2,5082019,1750.0,813.0,130406.0,2021988.0,431,2,5.0,...,19082019.0,NaN,NaN,E,NaN,NaN,NaN,NaN,NaN,NaN
1378903,1,2,10102019,2230.0,812.0,120060.0,25121920.0,498,1,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1378904,1,2,31052019,2000.0,816.0,160050.0,15011943.0,476,2,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
Mortalidade_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1349801 entries, 0 to 1349800
Data columns (total 87 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   ORIGEM      1349801 non-null  int64  
 1   TIPOBITO    1349801 non-null  int64  
 2   DTOBITO     1349801 non-null  int64  
 3   HORAOBITO   1292455 non-null  float64
 4   NATURAL     1276115 non-null  float64
 5   CODMUNNATU  1257268 non-null  float64
 6   DTNASC      1346010 non-null  float64
 7   IDADE       1349801 non-null  int64  
 8   SEXO        1349801 non-null  int64  
 9   RACACOR     1315348 non-null  float64
 10  ESTCIV      1274934 non-null  float64
 11  ESC         1245231 non-null  float64
 12  ESC2010     1239685 non-null  float64
 13  SERIESCFAL  314749 non-null   float64
 14  OCUP        1136322 non-null  float64
 15  CODMUNRES   1349801 non-null  int64  
 16  LOCOCOR     1349801 non-null  int64  
 17  CODESTAB    991068 non-null   float64
 18  ESTABDESCR  0 non-null

In [5]:
Mortalidade_2019.columns

Index(['ORIGEM', 'TIPOBITO', 'DTOBITO', 'HORAOBITO', 'NATURAL', 'CODMUNNATU',
       'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'ESC2010',
       'SERIESCFAL', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'CODESTAB', 'ESTABDESCR',
       'CODMUNOCOR', 'IDADEMAE', 'ESCMAE', 'ESCMAE2010', 'SERIESCMAE',
       'OCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEMAGESTAC',
       'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV',
       'OBITOPUERP', 'ASSISTMED', 'EXAME', 'CIRURGIA', 'NECROPSIA', 'LINHAA',
       'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'CAUSABAS', 'CB_PRE',
       'COMUNSVOIM', 'DTATESTADO', 'CIRCOBITO', 'ACIDTRAB', 'FONTE',
       'NUMEROLOTE', 'TPPOS', 'DTINVESTIG', 'CAUSABAS_O', 'DTCADASTRO',
       'ATESTANTE', 'STCODIFICA', 'CODIFICADO', 'VERSAOSIST', 'VERSAOSCB',
       'FONTEINV', 'DTRECEBIM', 'ATESTADO', 'DTRECORIGA', 'CAUSAMAT',
       'ESCMAEAGR1', 'ESCFALAGR1', 'STDOEPIDEM', 'STDONOVA', 'DIFDATA',
       'NUDIASOBCO', 'NUDIASOBIN', 'DTCADINV

# Fazendo Verificação e Limpeza da Tabela

In [6]:
Mortalidade_2019 = Mortalidade_2019.loc[:,['DTOBITO', 'CODMUNOCOR', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'CAUSABAS']]

Vendo os gêneros que mais sofreram com as mortes

In [7]:
Mortalidade_2019['SEXO'].value_counts()

1    745519
2    603725
0       557
Name: SEXO, dtype: int64

In [8]:
#Ao verificar que o Sexo que está igual a 1 possui maior valor, pode-se confirmar que são os homens, pois ao pesquisar sobre a mortalidade de gêneros, os homens são os que mais sofrem mortes

Corrigindo a coluna de idade

In [9]:
Mortalidade_2019['IDADE'] = Mortalidade_2019['IDADE'].astype(str)

def limpeza_idade(linha):
    x = linha['IDADE']
    if linha['DTOBITO']  == linha['DTNASC'] or linha['DTOBITO'] == linha['DTNASC']+1:
        return 0

    elif x.startswith('5') and len(x) == 3:
        x_new_5 = x.replace('5', '1', 1)
        x_new_5 = float(x)
        return x_new_5

    elif x.startswith('4') and (len(x) == 3 or len(x) == 2):
        x_new_4 = float(x[1:])
        return x_new_4

    elif x.startswith('3') and (len(x) == 3 or len(x) == 2):
        x_new_3 = float(x[1:])
        x_new_3 = (x_new_3 * 833334)/100
        return x_new_3

    elif x.startswith('2') and (len(x) == 3 or len(x) == 2):
        x_new_2 = float(x[1:])
        x_new_2 = (x_new_2 * 114155)/10000
        return x_new_2

    elif x.startswith('1') and (len(x) == 3 or len(x) == 2):
        x_new_1 = float(x[1:])
        x_new_1 = (x_new_1 * 1.90259)/1000000
        return x_new_1

Mortalidade_2019['IDADE'] = Mortalidade_2019.apply(limpeza_idade, axis = 1)

Mudando o formato das datas da coluna de óbito e nascimento

In [10]:
Mortalidade_2019['DTOBITO'] = Mortalidade_2019['DTOBITO'].astype(str).str.zfill(8)                          # deixa todas as linhas com 8 números, alem de transformar em str
Mortalidade_2019['DTOBITO'] = Mortalidade_2019['DTOBITO'].apply(lambda x: f'{x[0:2]}/{x[2:4]}/{x[4:]}')     # aplica '/' em todas as datas
Mortalidade_2019['DTOBITO'] = pd.to_datetime(Mortalidade_2019['DTOBITO'])                                   # transforma em datetime

In [11]:
Mortalidade_2019['DTNASC'] = Mortalidade_2019['DTNASC'].fillna(0)                      #transforma nulo em zeros
Mortalidade_2019['DTNASC'] = pd.to_datetime(Mortalidade_2019['DTNASC']).dt.date        #transforma em data
Mortalidade_2019['DTNASC'] = pd.to_datetime(Mortalidade_2019['DTNASC'])

Arrumando a coluna de Causas e classificando cada uma das siglas 

In [12]:
Mortalidade_2019['CAUSABAS'] = Mortalidade_2019['CAUSABAS'].apply(lambda x: f'{x[0:3]}')

In [13]:
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['CAUSABAS']

Classificação de causas (coluna CAUSABAS)

    - A00 ~ E99 --> doenças

    - F00       --> Transtornos Mentais

    - G00 ~ N99 --> doenças

    - O00 ~ Q99 --> problemas na gestão/Parto e malformações

    - R00       --> doenças

    - S00 ~ T98 --> Consequências de causas externas (lesões/ traumatismos)

    - U00 ~ Y98 --> Causas externas de morbidade e de mortalidade (acidente/agressao/intoxicação/falta de comida/falta de agua/queda/tiro)

    - Z00 ~     --> doenças

In [14]:
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[A-E].*', 'Doença')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('F.*', 'Transtornos Mentais')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[G-N].*', 'Doença')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[O-Q].*', 'problemas na gestão/Parto e malformações')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('R.*', 'Doença')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[S-T].*', 'Consequências de causas externas (lesões/ traumatismos)')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[U-Y].*', 'Causas externas de morbidade e de mortalidade (acidente/agressao)')
Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('Z.*', 'Doença')

C:\Users\Leonardo\AppData\Local\Temp/ipykernel_6892/1318979736.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[A-E].*', 'Doença')
C:\Users\Leonardo\AppData\Local\Temp/ipykernel_6892/1318979736.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('F.*', 'Transtornos Mentais')
C:\Users\Leonardo\AppData\Local\Temp/ipykernel_6892/1318979736.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  Mortalidade_2019['Classificação de causas'] = Mortalidade_2019['Classificação de causas'].str.replace('[G-N].*', 'Doença')
C:\Users\Leonardo\AppData\Local\Temp/ipykernel_6892/1318979736.py:4: FutureWarning: The default value of regex will change from

# Lendo arquivo CID para juntar as tabelas de CID e Mortalidade

In [15]:
CID_2019 = pd.read_excel(r'C:\Projeto Final\data\CID_2019.xlsx', header=15, skipfooter=8)

# Fazendo o tratamento do arquivo CID

In [16]:
CID_2019['Total'] = CID_2019['Total'].apply(lambda x: f'{x[0:3]}')
CID_2019 = CID_2019.loc[:,['Total']]
CID_2019  = CID_2019.drop(CID_2019.loc[CID_2019['Total']=='Cap'].index)
CID_2019['Classificação de causas'] = CID_2019['Total']

Classificação de causas (coluna CAUSABAS)

    - A00 ~ E99 --> doenças

    - F00       --> Transtornos Mentais

    - G00 ~ N99 --> doenças

    - O00 ~ Q99 --> problemas na gestão/Parto e malformações

    - R00       --> doenças

    - S00 ~ T98 --> Consequências de causas externas (lesões/ traumatismos)

    - U00 ~ Y98 --> Causas externas de morbidade e de mortalidade (acidente/agressao/intoxicação/falta de comida/falta de agua/queda/tiro)

    - Z00 ~     --> doenças

In [17]:
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[A-E].*', 'Doença')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('F.*', 'Transtornos Mentais')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[G-N].*', 'Doença')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[O-Q].*', 'problemas na gestão/Parto e malformações')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('R.*', 'Doença')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[S-T].*', 'Consequências de causas externas (lesões/ traumatismos)')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[U-Y].*', 'Causas externas de morbidade e de mortalidade (acidente/agressao)')
CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('Z.*', 'Doença')

C:\Users\Leonardo\AppData\Local\Temp/ipykernel_6892/4078995205.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[A-E].*', 'Doença')
C:\Users\Leonardo\AppData\Local\Temp/ipykernel_6892/4078995205.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('F.*', 'Transtornos Mentais')
C:\Users\Leonardo\AppData\Local\Temp/ipykernel_6892/4078995205.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  CID_2019['Classificação de causas'] = CID_2019['Classificação de causas'].str.replace('[G-N].*', 'Doença')
C:\Users\Leonardo\AppData\Local\Temp/ipykernel_6892/4078995205.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  CID_2019['

# Lendo Arquivo de Municípios para juntar com a tabela de mortalidade

In [18]:
Município_estabelecimento = pd.read_csv(r'C:\Projeto Final\data\cadastro_estabelecimentos_cnes.csv', sep = ';')

In [19]:
Município_estabelecimento = Município_estabelecimento.loc[:,['UF', 'IBGE']]

In [20]:
Município_estabelecimento = Município_estabelecimento.drop_duplicates()

juntando as tabelas

In [21]:
Mortalidade_Municípios_2019 = pd.merge(Mortalidade_2019, Município_estabelecimento, left_on = 'CODMUNOCOR', right_on ='IBGE')

# Lendo Arquivo de Estados para juntar na tabela

In [22]:
COD_UF = pd.read_excel(r'C:\Projeto Final\data\Tabela Estados IBGE.xlsx')

# Finalização da tabela limpa e verificada

In [23]:
Mortalidade_Estado_2019 = pd.merge(Mortalidade_Municípios_2019, COD_UF, left_on = 'UF', right_on ='Código da UF')

In [24]:
Mortalidade_Estado_2019 = Mortalidade_Estado_2019.loc[:, ['DTOBITO', 'CODMUNOCOR', 'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'CAUSABAS', 'Classificação de causas', 'Estado']]

In [25]:
Mortalidade_Estado_2019.to_csv(r'C:\Projeto Final\data\Mortalidade_Estado_2019.csv', sep = ';')